# Calculate monthly GC by Layers

In [67]:
monID = '12'
GClis = []
for path in Path('../OutputDirGFED').rglob('*Conc.2021' + str(monID) + '*.nc4'):
    GClis.append(path)
print(len(GClis))

31


In [ ]:
def map72to47(layer):
    if (layer<36):
        return layer
    elif (layer<45):
        return math.ceil((layer-36)/2)+35
    else:
        return math.ceil((layer-44)/4)+39

levels = list(gcDf['lev'].unique())
levToID = {}
for levID in range(len(levels)):
    levToID[levels[levID]] = 71-levID
    # levToID[levels[levID]] = levID
gcDf['levID'] = gcDf['lev'].map(levToID)

gcDf['levID47'] = gcDf['levID'].map(map72to47)
gcDf

In [68]:
count = 0
for DailyGC in GClis:
    metPath = str(DailyGC).replace('SpeciesConc','StateMet')

    if (count == 0):
        gc = xr.open_dataset(DailyGC, engine="netcdf4")
        tm = []
        for i in gc.time:
            if int(str(i)[47:49]) in [23, 0, 1, 2, 3, 4, 5, 6, 7, 8]:
                tm.append(str(i)[36:65])
        # gc['SpeciesConcMND_CH2O']
        met = xr.open_dataset(metPath, engine="netcdf4")['Met_BXHEIGHT']
        gc['Met_BXHEIGHT'] = met
        gc['VCD'] = gc['Met_BXHEIGHT']*gc['SpeciesConcMND_CH2O']*100
        # gcDf = gc.sel(time=tm)['SpeciesConcMND_CH2O', 'Met_BXHEIGHT']
        # .to_dataframe()
#         gcDf = gc.sel(time=tm)[['VCD','lev']].to_dataframe().reset_index().groupby(['time', 'lon','lat','lev']).sum().reset_index()
        gcDf = gc.sel(time=tm)[['VCD','lev']].to_dataframe().reset_index()
#     .groupby(['time', 'lon','lat','lev']).sum().reset_index()
        
        gcDf['hour'] = gcDf['time'].dt.hour
        # gcDf
#         tm = gcDf['time'].unique()
#         tm = list(tm)
#         tm2 = []
#         for a in tm:
#             tm2.append(str(a))        
    else:
        try:
            gcT = xr.open_dataset(DailyGC, engine="netcdf4")
            tm = []
            for i in gcT.time:
                if int(str(i)[47:49]) in [23, 0, 1, 2, 3, 4, 5, 6, 7, 8]:
                    tm.append(str(i)[36:65])
        # gc['SpeciesConcMND_CH2O']
        
            metT = xr.open_dataset(metPath, engine="netcdf4")['Met_BXHEIGHT']
            gcT['Met_BXHEIGHT'] = metT
            gcT['VCD'] = gcT['Met_BXHEIGHT']*gcT['SpeciesConcMND_CH2O']*100
            # gcDf = gc.sel(time=tm)['SpeciesConcMND_CH2O', 'Met_BXHEIGHT']
            # .to_dataframe()
            gcDfT = gcT.sel(time=tm)[['VCD','lev']].to_dataframe().reset_index()
#             .groupby(['time', 'lon','lat','lev']).sum().reset_index()
            gcDfT['hour'] = gcDfT['time'].dt.hour
            # gcDf
    #         tm = gcDf['time'].unique()
    #         tm = list(tm)
    #         tm2 = []
    #         for a in tm:
    #             tm2.append(str(a))
            gcDf = pd.concat([gcDf,gcDfT])
        except:
            print(metPath)
    
    count = count + 1
    print(count)
#     gcDf = gcDf.set_index(['time','lat','lon'])
#     gcDf = gcDf.to_xarray()
# hour = []
# for i in gcDf['time']:
#     hour.append(int(str(i)[-8:-6]))
# gcDf['hour'] = hour
# gcDf['Type'] = 'GC'
gcDf = gcDf.groupby(['hour', 'lon','lat','lev']).mean().reset_index()
levels = list(gcDf['lev'].unique())
levToID = {}
for levID in range(len(levels)):
    levToID[levels[levID]] = 71-levID
gcDf['levID'] = gcDf['lev'].map(levToID)

gcDf['levID47'] = gcDf['levID'].map(map72to47)

GCmon = gcDf[['lon', 'lat', 'hour','levID47', 'VCD']].groupby(['hour','lon','lat','levID47']).sum().reset_index()
GCmon.to_pickle('../GC_Pickle/GC2021' + str(monID) + 'Mon47')
GCmon

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


/tmp/ipykernel_2090/4274065082.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gcDf = gcDf.groupby(['hour', 'lon','lat','lev']).mean().reset_index()


,hour,lon,lat,levID47,VCD
0,0,60.000,-11.0,0,9.164027e+13
1,0,60.000,-11.0,1,9.293008e+13
2,0,60.000,-11.0,2,9.318047e+13
3,0,60.000,-11.0,3,9.323540e+13
4,0,60.000,-11.0,4,9.329066e+13
...,...,...,...,...,...
8933755,23,149.375,54.5,42,2.551945e+12
8933756,23,149.375,54.5,43,1.276118e+12
8933757,23,149.375,54.5,44,6.052001e+11
8933758,23,149.375,54.5,45,2.125312e+11


In [115]:
for DailyGC in GClis:
    try:
        metPath = str(DailyGC).replace('SpeciesConc','StateMet')

        gc = xr.open_dataset(DailyGC, engine="netcdf4")
        tm = []
        for i in gc.time:
            if int(str(i)[47:49]) in [23, 0, 1, 2, 3, 4, 5, 6, 7, 8]:
                tm.append(str(i)[36:65])
        met = xr.open_dataset(metPath, engine="netcdf4")['Met_BXHEIGHT']
        gc['Met_BXHEIGHT'] = met
        gc['VCD'] = gc['Met_BXHEIGHT']*gc['SpeciesConcMND_CH2O']*100
        gcDf = gc.sel(time=tm)[['VCD','lev']].to_dataframe().reset_index()        
        gcDf['hour'] = gcDf['time'].dt.hour
        # gcDf['month'] = gcDf['time'].dt.month
        # gcDf['day'] = gcDf['time'].dt.day

        levToID = {}
        for levID in range(len(levels)):
            levToID[levels[levID]] = 71-levID
        gcDf['levID'] = gcDf['lev'].map(levToID)

        gcDf['levID47'] = gcDf['levID'].map(map72to47)
        gcDf = gcDf[['time','lat','lon','VCD','hour','levID47']]
        gcDf.columns = ['time','latX','lonX','VCD','hour','layer']
        # if (monID != str(DailyGC)[-14:-12]):
        monID = str(DailyGC)[-14:-12]
        AK = pd.read_pickle('../GEMS_AK/GEMS_AKC2021' + monID)
        AK['hour'] = AK['hour'].astype(int)
        AK['hour'] = AK['hour'] + 1
        AK['hour'] = AK['hour'].replace(24,0)
        AK = AK.set_index(['layer','latX','lonX','hour'])
        gcDf = gcDf.join(AK, on = ['layer','latX','lonX','hour'], how='left')

        gcDf = gcDf.dropna()
        gcDf['VCD_AK']=gcDf['VCD']*gcDf['AK']

        gcDf = gcDf.reset_index()[['latX','lonX','hour','VCD_AK','VCD']].groupby(['latX','lonX','hour']).sum().reset_index()
        gcDf['month']=str(DailyGC)[-14:-12]
        gcDf['day']=str(DailyGC)[-12:-10]
        gcDf.to_pickle('../GC_AK_HCHO/GC_AK_HCHO_'+str(DailyGC)[-18:-10])
    except:
        print(str(DailyGC))

In [108]:
def dailyGCwithAK(DailyGC):
    try:
        metPath = str(DailyGC).replace('SpeciesConc','StateMet')

        gc = xr.open_dataset(DailyGC, engine="netcdf4")
        tm = []
        for i in gc.time:
            if int(str(i)[47:49]) in [23, 0, 1, 2, 3, 4, 5, 6, 7, 8]:
                tm.append(str(i)[36:65])
        met = xr.open_dataset(metPath, engine="netcdf4")['Met_BXHEIGHT']
        gc['Met_BXHEIGHT'] = met
        gc['VCD'] = gc['Met_BXHEIGHT']*gc['SpeciesConcMND_CH2O']*100
        gcDf = gc.sel(time=tm)[['VCD','lev']].to_dataframe().reset_index()        
        gcDf['hour'] = gcDf['time'].dt.hour
        # gcDf['month'] = gcDf['time'].dt.month
        # gcDf['day'] = gcDf['time'].dt.day

        levToID = {}
        for levID in range(len(levels)):
            levToID[levels[levID]] = 71-levID
        gcDf['levID'] = gcDf['lev'].map(levToID)

        gcDf['levID47'] = gcDf['levID'].map(map72to47)
        gcDf = gcDf[['time','lat','lon','VCD','hour','levID47']]
        gcDf.columns = ['time','latX','lonX','VCD','hour','layer']
        # if (monID != str(DailyGC)[-14:-12]):
        monID = str(DailyGC)[-14:-12]
        AK = pd.read_pickle('../GEMS_AK/GEMS_AKC2021' + monID)
        AK['hour'] = AK['hour'].astype(int)
        AK['hour'] = AK['hour'] + 1
        AK = AK.set_index(['layer','latX','lonX','hour'])
        gcDf = gcDf.join(AK, on = ['layer','latX','lonX','hour'], how='left')

        gcDf = gcDf.dropna()
        gcDf['VCD_AK']=gcDf['VCD']*gcDf['AK']

        gcDf = gcDf.reset_index()[['latX','lonX','hour','VCD_AK','VCD']].groupby(['latX','lonX','hour']).sum().reset_index()
        gcDf['month']=str(DailyGC)[-14:-12]
        gcDf['day']=str(DailyGC)[-12:-10]
        gcDf.to_pickle('../GC_AK_HCHO/GC_AK_HCHO_'+str(DailyGC)[-18:-10])
    except:
        print(str(DailyGC))

In [104]:
str(GClis[1])[-18:-10]

'20211201'

In [ ]:
Parallel(n_jobs=20)(delayed(dailyGCwithAK(iday)) for iday in GClis)